# <font color='GreenYellow' font-size:0.5em>Analiza wpływu czynników demograficznych i zawodowych na poziom dochodów w USA na podstawie zbioru danych "Adult"</font>

Autor: **Damian Opolski** \
Numer indeksu: **202280**

**Opis Problemu:**
Celem analizy jest zrozumienie, jak różne czynniki demograficzne i zawodowe wpływają na poziom dochodów dorosłych osób w Stanach Zjednoczonych. Analizujemy zależności między wiekiem, statusem zawodowym, edukacją, stanem cywilnym, zawodem, rasą, płcią oraz innymi zmiennymi a dwoma kategoriami dochodów: zarobki równiejsze lub mniejsze niż 50 tys. dolarów rocznie oraz zarobki wyższe niż 50 tys. dolarów rocznie.

**Źródło Danych:**
Dane użyte do analizy pochodzą z zestawu danych "Adult", który jest publicznie dostępny i często wykorzystywany w statystykach oraz pracach badawczych związanych z uczeniem maszynowym. Zestaw danych zawiera informacje na temat cech demograficznych, edukacyjnych, zawodowych i innych cech osób dorosłych, skupiając się na tych, którzy mogą być aktywni na rynku pracy w Stanach Zjednoczonych. Dane zostały zebrane w ramach spisu ludności, który dostarcza szerokiego przekroju danych, umożliwiających analizę wpływu tych cech na poziom dochodów.